In [ ]:
!pip install git+https://github.com/JihongJu/keras-resnet3d.git

In [ ]:
import os
from keras.optimizers import SGD
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from lib.data_loader import DataLoader
from lib.image import ImageDataGenerator
from lib.resnet_model import Resnet3DBuilder

In [ ]:
import os
os.chdir('C:/Users/Shubham/Hand-Gesture-Recognition-for-HCI-main/')

In [ ]:
root_directory = r'C:\Users\Shubham\Hand-Gesture-Recognition-for-HCI-main\dataset'
labels_csv_path = os.path.join(root_directory, 'labels_extracted.csv')
train_csv_path = os.path.join(root_directory, 'train_extracted.csv')
val_csv_path = os.path.join(root_directory, 'validation_extracted.csv')
train_path = os.path.join(root_directory, 'Train')
val_path = os.path.join(root_directory, 'Validation')

data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)

In [ ]:
data.train_df

In [ ]:
data.labels_df

In [ ]:
data.label_to_int

In [ ]:
WIDTH = 96
HEIGHT = 64
TARGET_SIZE = (HEIGHT, WIDTH)
BATCH_SIZE = 32
N_FRAMES = 16
SKIP = 1
INPUT_SHAPE = (N_FRAMES,) + TARGET_SIZE + (3,)
N_CLASSES = 10
DROP_RATE = 0.5
LEARNING_RATE = 0.01
EPOCHS = 10
WORKERS = 2

In [ ]:
model = Resnet3DBuilder.build_resnet_101(
    input_shape=INPUT_SHAPE, 
    num_outputs=N_CLASSES,
    drop_rate=DROP_RATE
)

model.summary()

In [ ]:
optimizer = SGD(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def train_model(model):

    image_datagen = ImageDataGenerator()

    train_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.train_df, 
        directory=train_path, 
        path_classes=labels_csv_path, 
        x_col='video_id', 
        y_col='label', 
        target_size=TARGET_SIZE, 
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE, 
        shuffle=True,
        seed=42,
        nb_frames=N_FRAMES, 
        skip=SKIP
    )

    validation_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.val_df, 
        directory=val_path, 
        path_classes=labels_csv_path, 
        x_col='video_id', 
        y_col='label', 
        target_size=TARGET_SIZE, 
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE, 
        shuffle=True,
        seed=42,
        nb_frames=N_FRAMES, 
        skip=SKIP
    )

    n_sample_train = len(data.train_df)
    n_sample_val = len(data.val_df)

    # model_checkpoint = ModelCheckpoint(
    #     'output/models/resnet_101_{epoch:02d}.h5', 
    #     period=1, 
    #     save_weights_only=False
    # )

    history = model.fit_generator(
        generator=train_generator, 
        steps_per_epoch=n_sample_train/BATCH_SIZE, 
        validation_data=validation_generator, 
        validation_steps=n_sample_val/BATCH_SIZE,
        epochs=1, 
        workers=WORKERS,
        verbose=1,
        # callbacks=[model_checkpoint]
    )

    model.save('output/models/resnet_101.h5')

    return history

In [ ]:
# epoch 1
history = train_model(model)

In [ ]:
# epoch 2
model = load_model('output/models/resnet_101_1.h5')
history = train_model(model)

In [ ]:
# epoch 3
model = load_model('output/models/resnet_101_2.h5')
history = train_model(model)

In [ ]:
# epoch 4 - 5
model = load_model('output/models/resnet_101_3.h5')
history = train_model(model)

In [ ]:
# epoch 6 - 7
model = load_model('output/models/resnet_101_5.h5')
history = train_model(model)